In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from typing import List, Tuple

/home/user/Project2_CourseRecommender/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pretrained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all courses
course_embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)
faiss.normalize_L2(course_embeddings)

NameError: name 'df' is not defined

In [ ]:
def recommend_courses(profile: str, completed_ids: List[str]) -> List[Tuple[str, float]]:
    filtered_df = df[~df['course_id'].isin(completed_ids)].copy()
    filtered_texts = filtered_df['text'].tolist()

    filtered_embeddings = model.encode(filtered_texts, show_progress_bar=False)
    faiss.normalize_L2(filtered_embeddings)

    dim = filtered_embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(filtered_embeddings)

    user_embedding = model.encode([profile])
    faiss.normalize_L2(user_embedding)

    scores, indices = index.search(user_embedding, k=5)

    results = []
    for i in range(5):
        idx = indices[0][i]
        score = scores[0][i]
        course_id = filtered_df.iloc[idx]['course_id']
        results.append((course_id, round(float(score), 4)))
        return results

In [ ]:
test_cases = [
    ("I’ve completed the ‘Foundations of Machine Learning’ course and enjoy data visualization.", ['C001']),
    ("I know Azure basics and want to manage containers and build CI/CD pipelines.", ['C006']),
    ("My background is in ML fundamentals; I’d like to specialize in neural networks and production workflows.", ['C001']),
    ("I want to learn to build and deploy microservices with Kubernetes—what courses fit best?", []),
    ("I’m interested in blockchain and smart contracts but have no prior experience.", [])
]

for profile, completed in test_cases:
    print(f"\nUser Query: {profile}")
    recommendations = recommend_courses(profile, completed)
    for course_id, score in recommendations:
        title = df[df['course_id'] == course_id]['title'].values[0]
        print(f"  - {course_id} ({score}): {title}")